In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import re

import pandas as pd


data = pd.read_csv("data.csv")
print(data)

          ID                                          Job Title  \
0          0  développeur full stack java  flutter confirmé ...   
1          1       consultant devops sénior  kubernetes grafana   
2          2  gestionnaire dapplication sénior  servicenow j...   
3          3       data scientist confirmé  servicenow bigquery   
4          4  développeur full stack java  vuejs gcp sénior ...   
...      ...                                                ...   
16463  16463                    product owner scurit api junior   
16464  16464                         dveloppeur back javamobile   
16465  16465  amoa souscription et gestion iard hors sinistr...   
16466  16466                              dveloppeur full stack   
16467  16467              ingnieure iamiga lille 3j par semaine   

      Publishing Date Start Date    Salary      Duration  \
0          2025-02-26       asap       450       24 mois   
1          2025-02-26       asap       500       12 mois   
2          2025

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [5]:
data.head()

,ID,Job Title,Publishing Date,Start Date,Salary,Duration,Location,Work Mode,Skills,Description,...,Company Name,Number of Candidates,Number of Employees,Sector,Description E,Experience,Contract Type,Education,scrap_date_parsed,estm_publishdate
0,0,développeur full stack java flutter confirmé ...,2025-02-26,asap,450,24 mois,lille,non spécifié,"java, flutter, kubernetes, tdd",nous sommes é la recherche dun développeur ful...,...,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,2025-02-26,2025-02-26
1,1,consultant devops sénior kubernetes grafana,2025-02-26,asap,500,12 mois,lille,non spécifié,"kubernetes, grafana, datadog",nou sommes é la recherche din consultant devop...,...,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,2025-02-26,2025-02-26
2,2,gestionnaire dapplication sénior servicenow j...,2025-02-26,asap,470,12 mois,lille,non spécifié,"servicenow, javascript",nous sommes é la recherche dun gestionnaire da...,...,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,2025-02-26,2025-02-26
3,3,data scientist confirmé servicenow bigquery,2025-02-26,asap,450,24 mois,lille,non spécifié,"servicenow, bigquery, data visualization",nous sommes é la recherche dun data scientist ...,...,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,2025-02-26,2025-02-26
4,4,développeur full stack java vuejs gcp sénior ...,2025-02-26,asap,450,24 mois,lille,non spécifié,"java, vue.js, gcp",nous sommes é la recherche dun développeur ful...,...,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,2025-02-26,2025-02-26


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16468 entries, 0 to 16467
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    16468 non-null  int64 
 1   Job Title             16468 non-null  object
 2   Publishing Date       16468 non-null  object
 3   Start Date            16468 non-null  object
 4   Salary                16468 non-null  object
 5   Duration              16468 non-null  object
 6   Location              16468 non-null  object
 7   Work Mode             16468 non-null  object
 8   Skills                16468 non-null  object
 9   Description           16468 non-null  object
 10  Plateforme            16468 non-null  object
 11  Scrap Date            16468 non-null  object
 12  Company Name          16468 non-null  object
 13  Number of Candidates  16468 non-null  object
 14  Number of Employees   16468 non-null  object
 15  Sector                16468 non-null

In [7]:
data["Education"].value_counts()


Education
non spécifié        14602
 bac +5 / master      999
 bac +2               455
 bac +4               214
 bac +3               197
 cap                    1
Name: count, dtype: int64

In [8]:
from sklearn.preprocessing import OneHotEncoder

# Make sure education is string type
data['Education'] = data['Education'].astype(str)

# Reshape and encode
edu_encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
encoded_education = edu_encoder.fit_transform(data['Education'].values.reshape(-1, 1))


In [9]:
print(encoded_education.shape)  # (n_samples, n_unique_education_levels)
print(encoded_education.toarray())  # Only for small datasets


(16468, 6)
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]


In [10]:
# Replace NaT with a fixed datetime value (e.g., Jan 1, 2000)
data['estm_publishdate'] = data['estm_publishdate'].fillna(pd.Timestamp('2025-01-01'))




In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16468 entries, 0 to 16467
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    16468 non-null  int64 
 1   Job Title             16468 non-null  object
 2   Publishing Date       16468 non-null  object
 3   Start Date            16468 non-null  object
 4   Salary                16468 non-null  object
 5   Duration              16468 non-null  object
 6   Location              16468 non-null  object
 7   Work Mode             16468 non-null  object
 8   Skills                16468 non-null  object
 9   Description           16468 non-null  object
 10  Plateforme            16468 non-null  object
 11  Scrap Date            16468 non-null  object
 12  Company Name          16468 non-null  object
 13  Number of Candidates  16468 non-null  object
 14  Number of Employees   16468 non-null  object
 15  Sector                16468 non-null

In [12]:
data["Work Mode"].value_counts()


Work Mode
télétravail partiel                                                 5706
non spécifié                                                        1943
télétravail fréquent                                                1378
télétravail non autorisé                                            1369
sur site temps plein cadre                                           650
                                                                    ... 
30 €k/an - 35 €k/an hybride temps plein confirmé                       1
35 €k/an - 45 €k/an hybride temps plein premier emploi                 1
38 €k/an - 48 €k/an sur site temps plein cadre                         1
842 €/mois - 2 158 €/mois sur site contrat stagiaire / alternant       1
50 $us/h à distance temps plein confirmé                               1
Name: count, Length: 284, dtype: int64

In [13]:
try:
    from sklearn.impute import SimpleImputer # Scikit-Learn 0.20+
except ImportError:
    from sklearn.preprocessing import Imputer as SimpleImputer

imputer = SimpleImputer(strategy="median")


In [14]:
data_num = data.drop('estm_publishdate', axis=1)

In [15]:
imputer.fit(data_num)

ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'développeur full stack java  flutter confirmé  java flutter'

In [ ]:
import numpy as np

def split_train_val_test(data, val_ratio=0.1, test_ratio=0.1):
    
    
    # Shuffle indices
    shuffled_indices = np.random.permutation(len(data))
    
    # Calculate sizes
    test_set_size = int(len(data) * test_ratio)
    val_set_size = int(len(data) * val_ratio)
    
    # Indices
    test_indices = shuffled_indices[:test_set_size]
    val_indices = shuffled_indices[test_set_size:test_set_size + val_set_size]
    train_indices = shuffled_indices[test_set_size + val_set_size:]
    
    # Return splits
    return (
        data.iloc[train_indices].reset_index(drop=True),
        data.iloc[val_indices].reset_index(drop=True),
        data.iloc[test_indices].reset_index(drop=True)
    )

train_set, val_set, test_set = split_train_val_test(data, val_ratio=0.15, test_ratio=0.15)

print("Train size:", len(train_set))
print("Validation size:", len(val_set))
print("Test size:", len(test_set))


Train size: 11528
Validation size: 2470
Test size: 2470


In [ ]:
data_cat = data[['Education']]
data_cat.head(10)



,Education
0,non spécifié
1,non spécifié
2,non spécifié
3,non spécifié
4,non spécifié
5,non spécifié
6,non spécifié
7,non spécifié
8,non spécifié
9,non spécifié


In [ ]:
try:
    from sklearn.preprocessing import OrdinalEncoder # just to raise an ImportError if Scikit-Learn < 0.20
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder # Scikit-Learn < 0.20

education_encoder = OneHotEncoder()
data_education_1hot = education_encoder.fit_transform(data_cat)
data_education_1hot

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 16468 stored elements and shape (16468, 6)>

In [ ]:
ordinal_encoder = OrdinalEncoder()
data_cat_encoded = ordinal_encoder.fit_transform(data_cat)
data_cat_encoded[:10]


[array([' bac +2', ' bac +3', ' bac +4', ' bac +5 / master', ' cap',
        'non spécifié'], dtype=object)]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16468 entries, 0 to 16467
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    16468 non-null  int64 
 1   Job Title             16468 non-null  object
 2   Publishing Date       16468 non-null  object
 3   Start Date            16468 non-null  object
 4   Salary                16468 non-null  object
 5   Duration              16468 non-null  object
 6   Location              16468 non-null  object
 7   Work Mode             16468 non-null  object
 8   Skills                16468 non-null  object
 9   Description           16468 non-null  object
 10  Plateforme            16468 non-null  object
 11  Scrap Date            16468 non-null  object
 12  Company Name          16468 non-null  object
 13  Number of Candidates  16468 non-null  object
 14  Number of Employees   16468 non-null  object
 15  Sector                16468 non-null

In [ ]:
data_education_1hot.toarray()

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]])

In [ ]:
data['estm_publishdate'] = pd.to_datetime(data['estm_publishdate'], errors='coerce')

median_date = data['estm_publishdate'].median()
data['estm_publishdate'].fillna(median_date, inplace=True)

date_cols = ['Publishing Date', 'Start Date', 'Scrap Date', 'scrap_date_parsed']
for col in date_cols:
    data[col] = pd.to_datetime(data[col], errors='coerce')

C:\Users\ahmed\AppData\Local\Temp\ipykernel_13956\3893840823.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['estm_publishdate'].fillna(median_date, inplace=True)
C:\Users\ahmed\AppData\Local\Temp\ipykernel_13956\3893840823.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[col] = pd.to_datetime(data[col], errors='coerce')
C:\Users\ahmed\AppData\Local\Temp\ipykernel_13956\38

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['salary_numeric'], bins=50)
plt.title('Processed Salary Distribution')
plt.xlabel('Salary')
plt.ylabel('Count')
plt.show()

ModuleNotFoundError: No module named 'seaborn'